In [1]:
!nvidia-smi

Wed May  1 14:11:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   64C    P8              18W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/jupyter/unattended-baggage-yolov8-modification


In [3]:
!pip install opencv-python
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.5 MB/s eta 0:00:00


In [1]:
!pip install -q \
clearml

In [13]:
!pip install -e .

Obtaining file:///home/jupyter/unattended-baggage-yolov8-modification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.2-0.editable-py3-none-any.whl size=16601 sha256=01a714cb67710d0bd4f7bed197a18bfae3defe2430d545d512f614a3413993b3
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-0uxqm91l/wheels/16/24/4e/77eb3faafdc0df8f4050b5c10cf91dc4af4801edeb0d66c251
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.2
    Uninstalling ultralytics-8.2.2:
      Successfully uninstalled ultralytics-8.2.2


## Setup ClearML

In [4]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
%env CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
env: CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4


## YOLOv8 baseline

In [5]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="1 May 2024 Model Training YOLOv8 baseline",
)

ClearML Task: created new task id=03cdd613e41a4de2ad14bfb18128491c
2024-05-01 02:56:36,982 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/03cdd613e41a4de2ad14bfb18128491c/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [6]:
# Step 2: Selecting the YOLOv8 Model, for this case baseline, yolov8n
model_variant = "yolov8n"
task.set_parameter("model_variant", model_variant)

In [7]:
# # Step 3: Loading the YOLOv8 Model
from ultralytics import YOLO
model = YOLO(f'{model_variant}.pt')

2024-05-01 02:57:49,148 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd


## YOLOv8 configuration used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size 
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for baseline training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_yolov8_baseline',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 4,
 'device': [0, 1],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_yolov8_baseline',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.5 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=1280, save=True, save_period=5, cache=False, device=[0, 1], workers=2, project=None, name=unattended_baggage_detection_yolov8_baseline, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 4 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.876      0.853      0.897      0.621

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      1.13G     0.7533     0.4267     0.9363          2       1280: 100%|██████████| 290/290 [00:45<00:00,  6.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 13.54it/s]


                   all        145        586      0.865      0.853      0.898      0.622

100 epochs completed in 1.386 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 12.81it/s]


                   all        145        586      0.878      0.857      0.906       0.63
              backpack        145        192      0.875      0.842      0.888      0.572
               handbag        145        310      0.844      0.872      0.911      0.526
              suitcase        145         84      0.916      0.857      0.919      0.793
Speed: 0.5ms preprocess, 13.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_yolov8_baseline
2024-05-01 04:29:13,604 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/1%20May%202024%20Model%20Training%20YOLOv8%20baseline.03cdd613e41a4de2ad14bfb18128491c/models/best.pt


## YOLOv8 + BilevelRoutingAttention

In [5]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="1 May 2024 Model Training BiLevelRouting Attention - Head",
)


ClearML Task: created new task id=e2cb7cdbb29a414a90520c804977596f
2024-05-01 14:12:43,381 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/e2cb7cdbb29a414a90520c804977596f/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [6]:
# Step 2: Selecting the YOLOv8 Model, for this case BilevelRoutingAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [7]:
# Step 3: Loading the YOLOv8 BilevelRoutingAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-01 14:13:00,174 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 84/361 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 BiLevelRouting Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_bilevelroutingattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_bilevelroutingattention_head',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 4,
 'device': [0, 1],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_bilevelroutingattention_head',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=1280, save=True, save_period=5, cache=False, device=[0, 1], workers=2, project=None, name=unattended_baggage_detection_bilevelroutingattention_head, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 67 weight(decay=0.0005), 66 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 4 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.867      0.688      0.795      0.519

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      15.3G     0.9813     0.7163      1.179          2       1280: 100%|██████████| 290/290 [01:35<00:00,  3.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:04<00:00,  7.79it/s]


                   all        145        586      0.826      0.704      0.788      0.516

100 epochs completed in 2.789 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head/weights/last.pt, 6.4MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head/weights/best.pt, 6.4MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
YOLOv8-BiLevelRoutingAttention summary (fused): 179 layers, 3056281 parameters, 0 gradients, 8.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 13.15it/s]


                   all        145        586      0.866      0.689      0.795       0.52
              backpack        145        192      0.873      0.734      0.778      0.467
               handbag        145        310      0.938       0.63       0.83      0.469
              suitcase        145         84      0.786      0.702      0.778      0.624
Speed: 0.5ms preprocess, 15.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_head
2024-05-01 17:02:22,488 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/1%20May%202024%20Model%20Training%20BiLevelRouting%20Attention%20-%20Head.e2cb7cdbb29a414a90520c804977596f/models/best.pt


## YOLOv8 + Shuffle Attention

In [ ]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="1 May 2024 Model Training Shuffle Attention",
)


In [ ]:
# Step 2: Selecting the YOLOv8 Model, for this case ShuffleAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [ ]:
# Step 3: Loading the YOLOv8 ShuffleAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

## YOLOv8 Shuffle Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_shuffleattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [ ]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=4,
                      device = [0,1],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_shuffleattention',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

In [ ]:
# Step 5: Initiating Model Training for ShuffleAttention
results = model.train(**args)